In [117]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Initial page exploration

In [84]:
url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

response = requests.get(url)

In [85]:
response.status_code

200

In [86]:
soup = BeautifulSoup(response.text)

In [87]:
soup.findAll('div', attrs = {'Members--list-item'})

# incumbant has 'a' tag, others have 'h2'
# peel off incumbant yes/no winner and affiliation with regex (flatten list by choosing index 0) 
# and then index[1] to get letter


[<div class="Members--list-item">
 <div class="Members--bio u-richtext">
 <h2><strong>
 <a href="/members-of-congress/mark-green/summary?cid=N00041873">Mark Green (R) • <i>Incumbent</i></a>
  • <span class="winner"><i>Winner</i></span> <br/>
 <span class="Members--vote-pct">(69.9% of vote)</span>
 </strong></h2>
 </div>
 <table border="0" cellpadding="1" cellspacing="0" class="Members--table">
 <tr>
 <td>Raised:</td>
 <td class="Members--number">$1,194,960</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:100.0%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Spent:</td>
 <td class="Members--number">$935,487</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:78.28599300862228%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Cash on Hand:</td>
 <td class="Members--number">$287,889</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar

In [94]:
s = soup.findAll('div', attrs = {'Members--list-item'})

In [95]:
s

[<div class="Members--list-item">
 <div class="Members--bio u-richtext">
 <h2><strong>
 <a href="/members-of-congress/mark-green/summary?cid=N00041873">Mark Green (R) • <i>Incumbent</i></a>
  • <span class="winner"><i>Winner</i></span> <br/>
 <span class="Members--vote-pct">(69.9% of vote)</span>
 </strong></h2>
 </div>
 <table border="0" cellpadding="1" cellspacing="0" class="Members--table">
 <tr>
 <td>Raised:</td>
 <td class="Members--number">$1,194,960</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:100.0%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Spent:</td>
 <td class="Members--number">$935,487</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:78.28599300862228%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Cash on Hand:</td>
 <td class="Members--number">$287,889</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar

In [127]:
# state, only do this once for each page
soup.find('h1', attrs = {'class': 'Hero-title'}).text.split()[0]

'Tennessee'

In [128]:
# district, only do this once for each page
soup.find('h1', attrs = {'class': 'Hero-title'}).text.split()[2]

'07'

In [ ]:
# for first candidate on page (only one with 'a' tag), all information
s[0].find('a').text.split(' (')[0] # candidate name
re.findall(r'\([A-Z]\)', s[0].find('a').text)[0][1] # party affiliation
s[0].find('a').text.split()[-1] # incumbant always with 'a' tag
s[0].find('span', attrs = {'class': 'winner'}).text # if present then winner
s[0].find('span', attrs = {'class': 'Members--vote-pct'}).text.strip('(').strip(')').replace('% of vote', '') # % of vote
s[0].findAll('td', attrs = {'class': 'Members--number'})[0].text.replace('$', '') # money raised
s[0].findAll('td', attrs = {'class': 'Members--number'})[1].text.replace('$', '') # money spent

In [136]:
# for all other candidates on page, put into loop with range of len(s)
s[1].find('strong').text.split(' (')[0].strip() # candidate name
re.findall(r'\([A-Z]\)', s[0].find('strong').text)[0][1] # party affiliation
s[1].find('span', attrs = {'class': 'winner'}).text # if present then winner
s[1].find('span', attrs = {'class': 'Members--vote-pct'}).text.strip('(').strip(')').replace('% of vote', '') # % of vote
s[1].findAll('td', attrs = {'class': 'Members--number'})[0].text.replace('$', '') # money raised
s[1].findAll('td', attrs = {'class': 'Members--number'})[1].text.replace('$', '') # money spent

'Kiran Sreepada'

In [177]:
s[1].find('strong').text.split(' (')[0].strip() # candidate name

'Kiran Sreepada'

In [185]:
# for single page

candidate_df = pd.DataFrame()
candidate_results = {}

state_name = soup.find('h1', attrs = {'class': 'Hero-title'}).text.split()[0]
district = soup.find('h1', attrs = {'class': 'Hero-title'}).text.split()[2]

candidates = soup.findAll('div', attrs = {'Members--list-item'})

for candidate in candidates:
    candidate_results['state_name'] = state_name
    candidate_results['district'] = district
    if candidate.find('a'):
        candidate_results['candidate_name'] = candidate.find('a').text.split(' (')[0]
        candidate_results['party'] = re.findall(r'\([A-Z]\)', candidate.find('a').text)[0][1]
        candidate_results['incumbent'] = candidate.find('a').text.split()[-1]
    else:
        candidate_results['candidate_name'] = candidate.find('strong').text.split(' (')[0].strip()
        candidate_results['party'] = re.findall(r'\([A-Z]\)', s[0].find('strong').text)[0][1]
        candidate_results['incumbent'] = 'not incumbent'
    if candidate.find('span', attrs = {'class': 'winner'}):
        candidate_results['winner'] = 1
    else:
        candidate_results['winner'] = 0
    candidate_results['perc_vote'] = candidate.find('span', attrs = {'class': 'Members--vote-pct'})\
        .text.strip('(').strip(')').replace('% of vote', '')
    candidate_results['money_raised'] = candidate.findAll('td', attrs = {'class': 'Members--number'})[0]\
        .text.replace('$', '')
    candidate_results['money_spent'] = candidate.findAll('td', attrs = {'class': 'Members--number'})[1]\
        .text.replace('$', '')
    can_res = pd.DataFrame([candidate_results])
    candidate_df = pd.concat([candidate_df, can_res])


In [186]:
candidate_df

,state_name,district,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent
0,Tennessee,07,Mark Green,R,Incumbent,1,69.9,"1,194,960","935,487"
0,Tennessee,07,Kiran Sreepada,R,not incumbent,0,27.3,"206,644","207,191"
0,Tennessee,07,Ronald Brown,R,not incumbent,0,2.2,"1,750",0
0,Tennessee,07,Scott Vieira Jr,R,not incumbent,0,0.6,655,"1,049"
